In [1]:
!pip install SoccerNet torch torchvision Linformer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 144.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 11.4 MB/s eta 0:00:00


In [2]:
import random
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import os, json, re
import numpy as np
from linformer import Linformer
from tqdm import tqdm
from collections import Counter
from SoccerNet.utils import getListGames


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Dataset Organization

In [4]:
LOCAL_DIR = "/content/drive/MyDrive/SoccerNetData"

class SoccerNetCaptionDataset(Dataset):
    def __init__(self, local_dir, split="train", feature_type="1_ResNET_TF2.npy", mode="random"):
        self.local_dir = local_dir
        self.feature_type = feature_type
        self.games = getListGames(split=split)
        self.mode = mode  # "random" or "concat"

    def __len__(self):
        return len(self.games)

    def __getitem__(self, idx):
        game = self.games[idx]
        game_dir = os.path.join(self.local_dir, game)
        label_path = os.path.join(game_dir, "Labels-v2.json")
        feature_path = os.path.join(game_dir, self.feature_type)

        if (not os.path.exists(feature_path)) or (not os.path.exists(label_path)):
            dummy_feat = torch.zeros((32, 2048), dtype=torch.float32)
            return dummy_feat, "No event"

        features = np.load(feature_path)
        features = torch.from_numpy(features).float()

        caption = "No event"
        try:
            with open(label_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            if "annotations" in data and len(data["annotations"]) > 0:
                if self.mode == "random":
                    ann = random.choice(data["annotations"])
                    event_label = ann.get("label", "Unknown")
                    team = ann.get("team", None)
                    minute = ann.get("gameTime", None)
                    caption = f"{event_label} by {team}" if team else event_label
                    if minute:
                        caption += f" at {minute}"

                elif self.mode == "concat":
                    events = []
                    for ann in data["annotations"][:5]:
                        label = ann.get("label", "Unknown")
                        team = ann.get("team", "")
                        minute = ann.get("gameTime", "")
                        events.append(f"{label} ({team}) at {minute}")
                    caption = " ; ".join(events)
        except Exception:
            caption = "No event"

        return features, caption

train_dataset = SoccerNetCaptionDataset(LOCAL_DIR, split="train", mode="random")
print(train_dataset[100][1])

val_dataset = SoccerNetCaptionDataset(LOCAL_DIR, split="valid", mode="random")
print(val_dataset[2][1])


Shots on target by away at 2 - 36:40
Direct free-kick by home at 1 - 37:43


### Light Encoder (Just to have the good dimensions)

In [5]:
class SoccerNetEncoder(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=512, pool_factor=4, dropout=0.1):
        super().__init__()

        self.proj = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout)
        )

        self.pool_factor = pool_factor

        self.temporal_gru = nn.GRU(hidden_dim, hidden_dim, batch_first=True, bidirectional=False)

        self.norm = nn.LayerNorm(hidden_dim)

    def forward(self, x):
        """
        x: [B, T, 2048] → video features
        """
        x = self.proj(x)  # [B, T, H]

        if x.size(1) > self.pool_factor:
            x = x.transpose(1, 2)  # [B, H, T]
            x = F.avg_pool1d(x, kernel_size=self.pool_factor, stride=self.pool_factor)
            x = x.transpose(1, 2)  # [B, T//pool_factor, H]

        x, _ = self.temporal_gru(x)

        x = self.norm(x)

        return x


### Decoder (GRU + Linformer)

In [6]:
class CaptionDecoder(nn.Module):
    def __init__(self, hidden_dim=512, vocab_size=10000, seq_len=64, n_heads=4, depth=3, linformer_k=64):
        super().__init__()

        self.linformer = Linformer(
            dim=hidden_dim,
            seq_len=seq_len,
            depth=depth,
            heads=n_heads,
            k=linformer_k
        )

        self.gru = nn.GRU(hidden_dim, hidden_dim, batch_first=True)

        self.fc = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, vocab_size)
        )

        self.max_seq_len = seq_len

    def forward(self, features):
        """
        features: [B, T, H]
        """
        if features.size(1) > self.max_seq_len:
            features = features[:, :self.max_seq_len, :]

        attn_out = self.linformer(features)  # [B, T, H]

        gru_out, _ = self.gru(attn_out)

        logits = self.fc(gru_out)  # [B, T, vocab_size]
        return logits


### Model : Encoder+Decoder

In [7]:
class RealTimeVideoCaptionNet(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=512, vocab_size=10000, seq_len=64):
        super().__init__()
        self.encoder = SoccerNetEncoder(input_dim, hidden_dim)
        self.decoder = CaptionDecoder(hidden_dim, vocab_size, seq_len)

    def forward(self, x):
        """
        x: [B, T, 2048]
        """
        encoded = self.encoder(x)
        captions = self.decoder(encoded)
        return captions

### Captions and Vocabulary

In [8]:
def collect_captions_from_games(local_dir, games, max_files=None):
    captions = []
    n = 0
    for game in games:
        label_path = os.path.join(local_dir, game, "Labels-v2.json")
        if not os.path.exists(label_path):
            continue

        try:
            with open(label_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            if "annotations" not in data or len(data["annotations"]) == 0:
                continue

            for ann in data["annotations"]:
                lbl = ann.get("label", "").strip()
                team = ann.get("team", None)
                time = ann.get("gameTime", None)
                half = ann.get("half", None)

                if lbl:
                    caption_parts = [lbl]
                    if team:
                        caption_parts.append(f"by the {team} team")
                    """
                    if time:
                        caption_parts.append(f"at {time}")
                    if half:
                        caption_parts.append(f"in half {half}")
                    """
                    caption = " ".join(caption_parts)
                    captions.append(caption)
                    n += 1

                    if max_files is not None and n >= max_files:
                        return captions
        except Exception:
            continue

    return captions

train_games = getListGames(split="train")
print(f"📂 Number of games (train): {len(train_games)}")

captions_sample = collect_captions_from_games(LOCAL_DIR, train_games, max_files=8000)
print(f"🗣️ Extracted captions: {len(captions_sample)}")
print("🔍 Example:", captions_sample[:5])


📂 Number of games (train): 300
🗣️ Extracted captions: 8000
🔍 Example: ['Kick-off by the away team', 'Ball out of play by the not applicable team', 'Throw-in by the away team', 'Ball out of play by the not applicable team', 'Corner by the away team']


### Tokenizer

In [9]:
class SimpleTokenizer:
    def __init__(self, captions, vocab_size=10000):
        words = []
        for c in captions:
            words += re.findall(r"\w+", c.lower())

        most_common = Counter(words).most_common(vocab_size - 4)
        self.itos = ["<PAD>", "<SOS>", "<EOS>", "<UNK>"] + [w for w, _ in most_common]
        self.stoi = {w: i for i, w in enumerate(self.itos)}

    def encode(self, text):
        if not isinstance(text, str) or not text.strip():
            return [self.stoi["<SOS>"], self.stoi["<EOS>"]]
        toks = ["<SOS>"] + re.findall(r"\w+", text.lower()) + ["<EOS>"]
        return [self.stoi.get(t, self.stoi["<UNK>"]) for t in toks]

    def decode(self, token_list):
        words = []
        for t in token_list:
            if t in (0, 1, 2):  # skip special tokens
                continue
            if t >= len(self.itos):
                words.append("<UNK>")
            else:
                words.append(self.itos[t])
        return " ".join(words)

if len(captions_sample) == 0:
    print("⚠️ No caption found")
    tokenizer = SimpleTokenizer(["no event"], vocab_size=1000)
else:
    tokenizer = SimpleTokenizer(captions_sample, vocab_size=10000)

vocab_size = len(tokenizer.itos)
print(f"🧾 Total Vocabulary: {vocab_size} words")
print("🧩 Examples :", tokenizer.itos[:34])

🧾 Total Vocabulary: 35 words
🧩 Examples : ['<PAD>', '<SOS>', '<EOS>', '<UNK>', 'by', 'the', 'team', 'away', 'home', 'ball', 'out', 'of', 'play', 'not', 'applicable', 'throw', 'in', 'kick', 'shots', 'target', 'foul', 'free', 'clearance', 'indirect', 'off', 'on', 'corner', 'substitution', 'direct', 'card', 'yellow', 'offside', 'goal', 'penalty']


###Data Loaders

In [10]:
def collate_fn(batch):
    """
    batch: list of (features [T_i, D], caption_str)
    Retour:
      features_padded: [B, max_T, D]
      targets_padded:  [B, max_L]
    """
    features_list, token_list = [], []

    for feat, cap in batch:
        if isinstance(feat, np.ndarray):
            feat = torch.from_numpy(feat).float()
        features_list.append(feat)
        toks = torch.tensor(tokenizer.encode(cap), dtype=torch.long)
        token_list.append(toks)

    features_padded = pad_sequence(features_list, batch_first=True)
    targets_padded = pad_sequence(token_list, batch_first=True, padding_value=tokenizer.stoi["<PAD>"])

    return features_padded, targets_padded

print("📦 Size of train_dataset:", len(train_dataset))
print("📦 Size of val_dataset:", len(val_dataset))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader   = torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

batch_features, batch_targets = next(iter(train_loader))
print("✅ Batch features shape:", batch_features.shape)
print("✅ Batch targets shape:", batch_targets.shape)
print("💬 Decoded example :", tokenizer.decode(batch_targets[0].tolist()))

📦 Size of train_dataset: 300
📦 Size of val_dataset: 100
✅ Batch features shape: torch.Size([4, 5400, 2048])
✅ Batch targets shape: torch.Size([4, 11])
💬 Decoded example : <UNK> <UNK>


### Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🚀 Device:", device)

model = RealTimeVideoCaptionNet(
    input_dim=2048,
    hidden_dim=512,
    vocab_size=vocab_size,
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.AdamW(model.decoder.parameters(), lr=1e-4)

EPOCHS = 5
best_val_loss = float("inf")

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]")

    for features, targets in pbar:
        features, targets = features.to(device), targets.to(device)
        outputs = model(features)
        min_len = min(outputs.size(1), targets.size(1))
        outputs = outputs[:, :min_len, :].reshape(-1, vocab_size)
        targets = targets[:, :min_len].reshape(-1)

        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

    avg_train_loss = train_loss / len(train_loader)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for features, targets in val_loader:
            features, targets = features.to(device), targets.to(device)
            outputs = model(features)
            min_len = min(outputs.size(1), targets.size(1))
            outputs = outputs[:, :min_len, :].reshape(-1, vocab_size)
            targets = targets[:, :min_len].reshape(-1)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    print(f"✅ Epoch {epoch+1}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), "best_decoder_linformer_gru_soccernet.pth")
        print("💾 New best model saved !")

torch.save(model.state_dict(), "decoder_linformer_gru_soccernet_final.pth")
print("💾 Final model : decoder_linformer_gru_soccernet_final.pth")

with open("tokenizer_vocab.json", "w", encoding="utf-8") as f:
    json.dump(tokenizer.itos, f, ensure_ascii=False, indent=2)
print("💾 Saved vocabulary : tokenizer_vocab.json")


🚀 Device: cuda


Epoch 1/5 [Train]: 100%|██████████| 75/75 [13:45<00:00, 11.00s/it, loss=1.6911]


✅ Epoch 1/5 | Train Loss: 1.7508 | Val Loss: 1.0222
💾 New best model saved !


Epoch 2/5 [Train]:  27%|██▋       | 20/75 [00:15<00:40,  1.36it/s, loss=1.1235]

### Model and Vocabulary Selection

In [14]:
VOCAB_PATH = "/content/tokenizer_vocab.json"
MODEL_PATH = "/content/best_decoder_linformer_gru_soccernet.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🚀 Device:", device)

model = RealTimeVideoCaptionNet(
    input_dim=2048,
    hidden_dim=512,
    vocab_size=vocab_size,
).to(device)

if not os.path.exists(VOCAB_PATH):
    raise FileNotFoundError(f"❌ Vocabulary not found : {VOCAB_PATH}")

with open(VOCAB_PATH, "r") as f:
    vocab_data = json.load(f)

if isinstance(vocab_data, list):
    idx2word = {i: w for i, w in enumerate(vocab_data)}
elif isinstance(vocab_data, dict):
    if "idx2word" in vocab_data:
        idx2word = {int(k): v for k, v in vocab_data["idx2word"].items()}
    elif all(k.isdigit() for k in vocab_data.keys()):
        idx2word = {int(k): v for k, v in vocab_data.items()}
    elif "itos" in vocab_data:
        idx2word = {i: w for i, w in enumerate(vocab_data["itos"])}
    elif "vocab" in vocab_data:
        idx2word = {i: w for i, w in enumerate(vocab_data["vocab"])}
    else:
        raise ValueError("❌ Non Recognized Vocabulary Format")
else:
    raise ValueError("❌ Unvalid Vocabulary Format")

vocab_size = len(idx2word)
print(f"📚 Vocabulary is downloaded : {vocab_size} words")
print("🧩 Examples :", list(idx2word.values())[:10])

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"❌ Model not found : {MODEL_PATH}")

state_dict = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(state_dict)
model.eval()

print("✅ Model and Vocabulary are successfully selected")


🚀 Device: cuda
📚 Vocabulary is downloaded : 98 words
🧩 Examples : ['<PAD>', '<SOS>', '<EOS>', '<UNK>', 'by', 'the', 'team', 'at', '2', '1']


RuntimeError: Error(s) in loading state_dict for RealTimeVideoCaptionNet:
	Missing key(s) in state_dict: "encoder.proj.0.weight", "encoder.proj.0.bias", "encoder.proj.1.weight", "encoder.proj.1.bias", "encoder.temporal_gru.weight_ih_l0", "encoder.temporal_gru.weight_hh_l0", "encoder.temporal_gru.bias_ih_l0", "encoder.temporal_gru.bias_hh_l0", "encoder.norm.weight", "encoder.norm.bias", "decoder.fc.0.weight", "decoder.fc.0.bias", "decoder.fc.1.weight", "decoder.fc.1.bias". 
	Unexpected key(s) in state_dict: "encoder.proj.weight", "encoder.proj.bias", "decoder.linformer.net.layers.3.0.fn.proj_k", "decoder.linformer.net.layers.3.0.fn.proj_v", "decoder.linformer.net.layers.3.0.fn.to_q.weight", "decoder.linformer.net.layers.3.0.fn.to_k.weight", "decoder.linformer.net.layers.3.0.fn.to_v.weight", "decoder.linformer.net.layers.3.0.fn.to_out.weight", "decoder.linformer.net.layers.3.0.fn.to_out.bias", "decoder.linformer.net.layers.3.0.norm.weight", "decoder.linformer.net.layers.3.0.norm.bias", "

In [ ]:

# ==========================================================
# ⚙️ Chargement des features
# ==========================================================
game_path = "/content/drive/MyDrive/SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley"
feature_path = os.path.join(game_path, "1_ResNET_TF2.npy")

if not os.path.exists(feature_path):
    raise FileNotFoundError(f"❌ Fichier introuvable : {feature_path}")

print(f"📂 Chargement des features depuis : {feature_path}")

features = np.load(feature_path)
features = torch.tensor(features, dtype=torch.float32).unsqueeze(0).to(device)  # [1, T, 2048]

print(f"✅ Features chargées : {features.shape}")

# ==========================================================
# 🧩 Segmentation en sous-séquences
# ==========================================================
max_len = 64
segments = []

for i in range(0, features.shape[1], max_len):
    seg = features[:, i:i+max_len, :]
    # Padding à la fin si nécessaire
    if seg.shape[1] < max_len:
        pad = torch.zeros(1, max_len - seg.shape[1], features.shape[2]).to(device)
        seg = torch.cat([seg, pad], dim=1)
    segments.append(seg)

print(f"📏 Nombre de segments : {len(segments)}")

# ==========================================================
# 🧠 Inférence
# ==========================================================
predicted_tokens = []

model.eval()
with torch.no_grad():
    for seg in segments:
        logits = model(seg)  # [1, T, vocab_size]
        preds = torch.argmax(logits, dim=-1)  # [1, T]
        predicted_tokens.extend(preds.squeeze(0).tolist())

# ==========================================================
# 🗣️ Reconstruction de la légende
# ==========================================================
caption_words = [
    idx2word.get(t, "<UNK>")
    for t in predicted_tokens
    if t not in [0, 1, 2, 3]  # ignore <PAD>, <SOS>, <EOS>, <UNK>
]

# Nettoyage simple : éviter répétitions directes et tokens vides
clean_caption = []
for w in caption_words:
    if not clean_caption or w != clean_caption[-1]:
        clean_caption.append(w)

caption_text = " ".join(clean_caption).replace("<UNK>", "").strip()

print("\n🏟️ Caption générée :")
print("🗨️", caption_text if caption_text else "(vide ou non décodable)")


📂 Chargement des features depuis : /content/drive/MyDrive/SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_ResNET_TF2.npy
✅ Features chargées : torch.Size([1, 5400, 2048])
📏 Nombre de segments : 85

🏟️ Caption générée :
🗨️ foul by at 1 throw in by at 1 foul in by at 1 throw by at 1 throw in by at 1 throw in by at 1 throw in by at 1 throw in by at 1 ball by at 1 throw by at 1 throw by at 1 throw in by at 1 applicable throw in by at 1 throw by at 1 foul in by at 1 throw in by at 1 foul in by at 1 throw in by at 1 ball by at 1 foul in by at 1 throw in by at 1 ball by at 1 ball by at 1 throw in by at 1 foul in by at 1 ball by at 1 throw in by at 1 throw in by at 1 throw in by at 1 throw in by at 1 throw in by at 1 foul by at 1 throw in by at 1 throw in by at 1 throw by at 1 ball by at 1 throw in by at 1 throw in by at 1 throw in by at 1 throw in by at 1 throw in by at 1 throw in by at 1 throw in by at 1 throw in by at 1 throw in by at 1 foul in by at 1 ball in

In [ ]:
print(tokenizer.itos)

['<PAD>', '<SOS>', '<EOS>', '<UNK>', 'by', 'the', 'team', 'at', '2', '1', 'away', 'home', 'ball', 'out', 'of', 'play', 'not', 'applicable', 'throw', 'in', 'kick', 'shots', 'target', 'foul', 'free', 'clearance', 'indirect', 'off', '00', 'on', 'corner', '24', '05', '04', '29', '25', '39', '03', '41', '35', '10', '43', '16', '34', '42', '09', '37', '27', '13', '14', '40', '11', '20', '02', '30', '22', '28', '21', '19', '01', '31', '08', '15', '17', '23', '12', '06', '33', '36', '18', '26', '44', '38', '32', '07', '45', 'substitution', '47', '46', 'direct', '49', '48', 'card', 'yellow', '51', '55', '53', '57', '59', '52', '58', '56', '50', 'offside', '54', 'goal', 'penalty', 'red']


In [ ]:
import torch
import numpy as np
import os
import re
import json
import torch.nn.functional as F

# ==========================================================
# ⚙️ Chargement du modèle et du vocabulaire
# ==========================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Charger vocabulaire ---
vocab_path = "/content/tokenizer_vocab.json"
with open(vocab_path, "r") as f:
    vocab_data = json.load(f)

# compatibilité : différents formats possibles
if isinstance(vocab_data, dict):
    if "idx2word" in vocab_data:
        idx2word = {int(k): v for k, v in vocab_data["idx2word"].items()}
    elif all(isinstance(k, str) for k in vocab_data.keys()):
        idx2word = {int(k): v for k, v in vocab_data.items()}
    else:
        raise ValueError("Format de vocab invalide.")
else:
    raise ValueError("Fichier vocabulaire inattendu.")

vocab_size = len(idx2word)
print(f"📖 Vocabulaire chargé ({vocab_size} tokens)")

# --- Charger le modèle ---
model = RealTimeVideoCaptionNet(
    input_dim=2048,
    hidden_dim=512,
    vocab_size=vocab_size,
    reduce_factor=100
).to(device)

model.load_state_dict(torch.load("/content/best_decoder_linformer_gru_soccernet.pth", map_location=device))
model.eval()

print("✅ Modèle chargé et prêt pour l’inférence")

# ==========================================================
# ⚽ Chargement et réduction des features
# ==========================================================
game_path = "/content/drive/MyDrive/SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley"
feature_path = os.path.join(game_path, "1_ResNET_TF2.npy")

print(f"📂 Chargement des features depuis : {feature_path}")
features = np.load(feature_path)
features = torch.tensor(features, dtype=torch.float32).unsqueeze(0).to(device)  # [1, T, 2048]
print("✅ Features shape :", features.shape)

# --- Réduction temporelle dynamique (au lieu de 85 segments séparés) ---
def reduce_features(x, factor=100):
    B, T, D = x.shape
    T_new = T // factor
    x = x[:, :T_new * factor, :]  # découpage propre
    x = x.view(B, T_new, factor, D).mean(dim=2)  # moyenne temporelle par bloc
    return x

reduced_feats = reduce_features(features, factor=64)  # réduire ≈ 64x
print("📉 Features réduites :", reduced_feats.shape)

# ==========================================================
# 🧠 Prédiction avec Top-k Sampling
# ==========================================================
def top_k_sampling(logits, k=5):
    """Choisir un token via top-k sampling (plus naturel que argmax)."""
    probs = F.softmax(logits / 1.0, dim=-1)  # temp=1.0
    topk_probs, topk_indices = torch.topk(probs, k)
    sampled = torch.multinomial(topk_probs, num_samples=1)
    next_token = topk_indices.gather(-1, sampled)
    return next_token.item()

predicted_tokens = []

with torch.no_grad():
    logits = model(reduced_feats)  # [1, seq_len, vocab]
    for t in range(logits.shape[1]):
        token_id = top_k_sampling(logits[0, t], k=5)
        predicted_tokens.append(token_id)

# ==========================================================
# 🧹 Nettoyage de la sortie
# ==========================================================
caption_words = [
    idx2word.get(t, "<UNK>")
    for t in predicted_tokens if t not in [0, 1, 2, 3]
]

# suppression de répétitions excessives
clean_caption = []
for w in caption_words:
    if not clean_caption or clean_caption[-1] != w:
        clean_caption.append(w)

caption_text = " ".join(clean_caption)
caption_text = re.sub(r"\s+", " ", caption_text).strip()

print("\n🏟️ Caption générée :")
print("🗨️", caption_text)
